Задание 1

In [34]:
import pandas as pd

In [35]:
movies_data = pd.read_csv("ml-latest-small/movies.csv")
movies_ratings = pd.read_csv("ml-latest-small/ratings.csv")

In [36]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [37]:
movies_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [38]:
%%time

def movies_classification(movie_row):
    if movie_row['rating'] <= 2.0:
        return 'low'
    elif movie_row['rating'] <= 4.0:
        return 'mid'
    else:
        return 'high'


movies_ratings['rating_class'] = movies_ratings.apply(movies_classification, axis=1)


CPU times: total: 438 ms
Wall time: 865 ms


In [39]:
movies_ratings.head()

,userId,movieId,rating,timestamp,rating_class
0,1,31,2.5,1260759144,mid
1,1,1029,3.0,1260759179,mid
2,1,1061,3.0,1260759182,mid
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,mid


In [40]:
result_frame = (
    pd.merge(movies_data, movies_ratings)
    .groupby(["movieId", "title"])
    .agg({"rating": "mean", "rating_class": [set]})
    .reset_index()
)
result_frame.head()

,movieId,title,rating,rating_class
,,,mean,set
0,1,Toy Story (1995),3.872470,"{mid, low, high}"
1,2,Jumanji (1995),3.401869,"{mid, low, high}"
2,3,Grumpier Old Men (1995),3.161017,"{mid, low, high}"
3,4,Waiting to Exhale (1995),2.384615,"{mid, low}"
4,5,Father of the Bride Part II (1995),3.267857,"{mid, low, high}"


Задание 2

In [41]:
keywords_data = pd.read_csv("ml-latest-small/keywords.csv")

geo_data = {
    "Центр": ["москва", "тула", "ярославль"],
    "Северо-Запад": ["петербург", "псков", "мурманск"],
    "Дальний Восток": ["владивосток", "сахалин", "хабаровск"],
}
keywords_data.head(5)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [44]:
%%time


def region_detector(keyword_row):
    keywords_line = keyword_row.strip().split()
    for region, cities in geo_data.items():
        for keyword in keywords_line:
            if keyword.lower() in cities:
                return region
    return "undefined"


keywords_data["region"] = keywords_data["keyword"].apply(region_detector)

keywords_data.head()

CPU times: total: 141 ms
Wall time: 300 ms


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [45]:
keywords_data[keywords_data.region != "undefined"].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
